In [1]:
from google.colab import drive
drive.mount('/content/drive')

#  Navigate to project folder
%cd /content/drive/MyDrive/'Perceval Quest - Naples Quantum2Pi Knights'/'Phase1 - Naples Quantum2Pi Knights'/data

#List files to confirm location
!ls

Mounted at /content/drive
/content/drive/MyDrive/Perceval Quest - Naples Quantum2Pi Knights/Phase1 - Naples Quantum2Pi Knights/data
train.csv  val.csv


In [2]:
!pip install torch torchvision   #torch==2.* torchvision~=0.19
!pip install matplotlib pandas tqdm==4.67.1   #pandas==2.2.3    jupyter
!pip install perceval-quandela #!pip install perceval-quandela==0.12.0  0.11.2
!pip install hyperopt   # Only if you're using TPE
!pip install sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [56]:
import os
import torch
import yaml  # if needed; otherwise, we define config inline
import numpy as np

# Instead of loading a YAML file, define the config dictionary inline:
cfg = {
    "data": {
        "train_csv": "data/train.csv",
        "test_csv": "data/val.csv",
        "batch_size": 30,
        "shuffle": True
    },
    "model": {
        "type": "hybrid",  # use quantum embedding; set to "classical" for baseline
        "fourier_features": 50,
        "conv_channels": 16,
        "kernel_size": 5,
        "output_dim": 30
    },
    "quantum": {
        "backend": "local",          # use "local" simulation
        "n_modes": 24,
        "n_photons": 10,
        "postselect": 3,
        "n_samples": 1000,
        "circuit_type": "pdf",       # Options: "triangular", "rectangular", "pdf", "base"
        "variational": True,         # set to False for non-variational version
        "adaptive_sampling": False,
        "cache_enabled": True,
        "cache_directory": "results/cache",
        "local_backend": "SLOS",
        "scaleway_platform": "sim:sampling:2l4",
        "downsample_method": "pca"   # or "bilinear"
    },
    "training": {
        "epochs": 5,
        "learning_rate": 0.001,
        "optimizer": "adam",         # "adam" or "sgd"
        "device": "auto"             # "cuda", "cpu", or "auto"
    },
    "optimization": {
        "enabled": True,
        "method": "tpe",  # "tpe", "grid", or "optuna"
        "max_evals": 10,
        "space": {
            "n_modes": [6, 12, 24, 30],
            "n_photons": [6, 10],
            "learning_rate": [0.0001, 0.001, 0.01]
        }
    }
}

# You may also need to ensure that your "data" and "results" folders exist:
os.makedirs("data", exist_ok=True)
os.makedirs("results", exist_ok=True)
os.makedirs("results/cache", exist_ok=True)


In [57]:
import torch
print("CUDA available:", torch.cuda.is_available())

CUDA available: True


In [58]:
import torch
from tqdm import tqdm
import time
import torch.nn.functional as F
import os
import pickle

# Get the directory of this file (training.py)
base_dir = os.getcwd()
# Go up one level (assuming your project structure has 'src' and 'data' as siblings)
data_dir = os.path.join(base_dir, "data")
pca_model_path = os.path.join(data_dir, "pca_model.pkl")


def fit(epochs, lr, model, train_loader, val_loader, bs, optimizer, cfg):
    """
    Train the model for a specified number of epochs.
    If the model uses quantum embedding (embedding_size > 0), embed images using the BosonSampler.
    """
    history = []
    train_loss_list, train_acc_list, val_loss_list, val_acc_list = [], [], [], []


    # Compute the downsample shape (e.g. (9, 14) if nb_parameters == 126)
    downsample_shape = compute_downsample_shape(bs.nb_parameters)
    print(f"Downsample shape for images set to: {downsample_shape[0]}x{downsample_shape[1]}")

    for epoch in range(epochs):
        model.train()
        running_loss, running_acc = 0.0, 0.0
        for step, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")):
            if model.embedding_size:
                # Assume batch contains a list of images if batch_size > 1.
                images, labs = batch  # images shape: [B, 1, 28, 28]
                # Process each image sequentially (QaaS does not handle batching well)
                emb_list = []
                for img in images:

                    if cfg["quantum"].get("downsample_method", "bilinear") == "pca":
                        # Load your pre-trained PCA model (ensure you have it saved in pca_model.pkl)
                        with open(pca_model_path, "rb") as f:
                            pca_model = pickle.load(f)
                        # Use the PCA-based downsampling and assign to img_resized
                        img_resized  = pca_downsample(img, pca_model)
                    else:
                        # Use bilinear interpolation as fallback
                        # Using sequential embedding (remove parallel_embed call)
                        # Add batch and channel dimensions: [1, 1, 28, 28]
                        img_expanded = img.unsqueeze(0)  # now shape: [1, 1, 28, 28]
                        # Resize image to computed downsample shape.  (e.g. (9,14))
                        img_resized = F.interpolate(img_expanded, size=downsample_shape, mode='bilinear', align_corners=False)
                        # Remove batch and channel dims: shape becomes [height, width]  resulting shape becomes [1, 9, 14]
                        img_resized = img_resized.squeeze(0)
                        # Now the flattened tensor has exactly downsample_shape[0]*downsample_shape[1] pixels.

                    emb = bs.embed(img_resized, bs.n_samples)
                    emb_list.append(emb)

                # Stack embeddings and add the batch dimension if needed.
                #embeddings = torch.stack(embeddings)  # shape: [B, embedding_size]
                emb = torch.stack(emb_list)

                # TODO concatenate Fourier features
                #  # Compute Fourier features for each image and stack them:
                # fourier_feats = [fourier_encode(img, n_features=cfg["model"].get("fourier_features", 50)) for img in image_list]
                # fourier_feats = torch.stack(fourier_feats)  # shape: [B, fourier_features]

                # # Concatenate along feature dimension:
                # emb = torch.cat((emb, fourier_feats), dim=1)  # new embedding size = bs.embedding_size + fourier_features


                # Now call the training step (ensure your model.training_step accepts batched embeddings)
                loss, acc = model.training_step((images, labs), emb=emb)
                #loss, acc = model.training_step(batch, emb=emb)
            else:
                loss, acc = model.training_step(batch)

            #if step % 50 == 0:
            tqdm.write(f"Step {step}: loss={loss.item():.4f}, acc={acc.item():.4f}")

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            running_loss += float(loss.detach())
            running_acc += float(acc.detach())
            if model.embedding_size and step % 100 == 0 and step > 0:
                print(f"Step {step}, Training acc: {running_acc/(step+1):.4f}, Loss: {running_loss/(step+1):.4f}")
        result = evaluate(model, val_loader, bs, cfg)
        model.epoch_end(epoch, result)
        history.append(result)
        train_loss_list.append(running_loss / len(train_loader))
        train_acc_list.append(running_acc / len(train_loader))
        val_loss_list.append(result["val_loss"])
        val_acc_list.append(result["val_acc"])
        plot_training_metrics(train_acc_list, val_acc_list, train_loss_list, val_loss_list)
    return history



#TODO Integrate Fourier Features into the Data Pipeline  **
#  Concatenate with Quantum Embedding. OR  Preprocess Images Before Feeding into the CNN.
#fourier_feats = fourier_encode(images.squeeze(0).squeeze(0), n_features=50)  # choose 50 features
## After obtaining the quantum embedding (emb), concatenate them:
#emb = torch.cat((emb, fourier_feats.unsqueeze(0)), dim=1)
#######
#fourier_size = cfg["model"].get("fourier_features", 0)
#total_embedding_size = bs.embedding_size + fourier_size if cfg["model"]["type"] == "hybrid" else 0
## When initializing HybridMnistModel:
#net = HybridMnistModel(device=device, embedding_size=total_embedding_size)




def evaluate(model, val_loader, bs, cfg):
    """
    Evaluate the model on the validation set.
    If quantum embedding is used, compute it for each sample.
    """
    model.eval()
    outputs = []

    # Compute the target downsample shape (e.g. 9x14 if bs.nb_parameters equals 126)
    downsample_shape = compute_downsample_shape(bs.nb_parameters)

    with torch.no_grad():
        for batch in val_loader:
            if model.embedding_size:
                images, labs = batch # images shape: [B, 1, 28, 28]
                batch_embeddings = []
                # Process each image in the batch individually:
                for i in range(images.size(0)):
                    img = images[i]  # shape: [1, 28, 28]
                    # Apply downsampling based on the chosen method.
                    if cfg["quantum"].get("downsample_method", "bilinear") == "pca":
                        # Load the PCA model (ensure the path is correct)
                        with open(pca_model_path, "rb") as f:
                            pca_model = pickle.load(f)
                        img_resized = pca_downsample(img, pca_model)
                    else:
                        # Use bilinear interpolation
                        # Expand dimensions to [1, 1, 28, 28] if needed.
                        img_expanded = img.unsqueeze(0)
                        img_resized = F.interpolate(img_expanded, size=downsample_shape, mode='bilinear', align_corners=False)
                        img_resized = img_resized.squeeze(0)


                    # Now img_resized should have exactly downsample_shape[0]*downsample_shape[1] pixels.
                    # Compute the quantum embedding.
                    # Depending on configuration, use adaptive or standard embedding
                    if cfg["quantum"].get("adaptive_sampling", False):
                        emb = bs.adaptive_embed(img_resized, min_samples=100, max_samples=bs.n_samples, tol=1e-3)
                    else:
                        emb = bs.embed(img_resized, bs.n_samples)
                    batch_embeddings.append(emb)
                # Stack embeddings to form a tensor of shape [B, embedding_size]
                emb_tensor = torch.stack(batch_embeddings)
                # Pass the batch and the corresponding embeddings to the model's validation step
                out = model.validation_step((images, labs), emb=emb_tensor)
            else:
                out = model.validation_step(batch)
            outputs.append(out)
    batch_losses = [x["val_loss"] for x in outputs]
    batch_accs = [x["val_acc"] for x in outputs]
    val_loss = torch.stack(batch_losses).mean().item()
    val_acc = torch.stack(batch_accs).mean().item()
    return {"val_loss": val_loss, "val_acc": val_acc}

In [59]:
# src/boson_sampler.py
import perceval as pcvl
from perceval.providers import scaleway  # import Scaleway Session class if needed

import torch
import torch.nn as nn
import os
import hashlib
import pickle
import numpy as np
from scipy.signal import convolve2d

from abc import ABC, abstractmethod


def merge_circuits(c1, c2):
    """
    Merge two circuits by creating a new composite circuit.
    This helper creates a new circuit with the same number of modes as c1,
    and adds c1 and then c2 using the add() method with merge=True.
    """
    composite = pcvl.Circuit(c1.m)
    # Add the first circuit on port 0 and merge it into the composite circuit.
    composite.add(0, c1, merge=True)
    # Add the second circuit similarly.
    composite.add(0, c2, merge=True)
    return composite

def compose_circuits(circuit_list):
    """
    Compose a list of circuits into a single circuit using merge_circuits.
    """
    composite = circuit_list[0]
    for circ in circuit_list[1:]:
        composite = merge_circuits(composite, circ)
    return composite




class InterferometerBuilder(ABC):
    @abstractmethod
    def create_circuit(self, m: int, parameters: list[float] = None) -> pcvl.Circuit:
        """Return a Perceval circuit for m modes given a list of phase parameters."""
        pass

def safe_param(i, params):
    # If i is within bounds, return the parameter; otherwise, default to 0.
    return params[i] if i < len(params) else 0

class TriangularInterferometerBuilder(InterferometerBuilder):
    def create_circuit(self, m: int, parameters: list[float] = None) -> pcvl.Circuit:
        if parameters is None:
            # Use symbolic parameters: for a triangular mesh you need m*(m-1) parameters.
            parameters = [pcvl.P(f"phi_{i}") for i in range(m * (m - 1))]
        # Build the circuit using the local variable m and the correct variable name (parameters)
        return pcvl.GenericInterferometer(m, lambda idx: (
            pcvl.BS()
            .add(0, pcvl.PS(safe_param(2 * idx, parameters)))
            .add(0, pcvl.BS())
            .add(0, pcvl.PS(safe_param(2 * idx + 1, parameters)))
        ))


class RectangularInterferometerBuilder(InterferometerBuilder):
    def create_circuit(self, m: int, parameters: list[float] = None) -> pcvl.Circuit:
        if parameters is None:
            parameters = [pcvl.P(f"phi_rect_{i}") for i in range(m * (m - 1))]
        # For demonstration, we build two layers sequentially.
        # (Note: In practice, you’d implement the proper rectangular mesh.)
        layer1 = pcvl.GenericInterferometer(m, lambda idx: (
            pcvl.BS().add(0, pcvl.PS(safe_param(2 * idx, parameters)))
        ))
        layer2 = pcvl.GenericInterferometer(m, lambda idx: (
            pcvl.BS().add(0, pcvl.PS(safe_param(2 * idx + 1, parameters)))
        ))
        return merge_circuits(layer1, layer2)



class PdfInterferometerBuilder:
    def create_circuit(self, m: int, parameters=None) -> pcvl.Circuit:
        """
        Create a circuit as described in Quantum_Circuits.pdf.
        Expects 'parameters' to be either:
          - A dict with keys 'theta', 'alpha', and 'beta'
            where 'theta' is a list of three phase values,
                  'alpha' is a list of two values for the first BS block, and
                  'beta' is a list of two values for the second BS block,
          - Or a list of at least 7 numbers, which will be interpreted as
            theta[0:3], alpha[0:2], beta[0:2].
        """
        if parameters is None:
            parameters = {
                'theta': [pcvl.P("theta1"), pcvl.P("theta2"), pcvl.P("theta3")],
                'alpha': [0.1, 0.2],
                'beta': [0.3, 0.4]
            }
        elif not isinstance(parameters, dict):
            # Assume parameters is a list; require at least 7 values
            if len(parameters) < 7:
                raise ValueError("PDFInterferometerBuilder requires at least 7 parameters when provided as a list.")
            parameters = {
                'theta': parameters[:3],
                'alpha': parameters[3:5],
                'beta': parameters[5:7]
            }

        def U_block(m, theta):
            sub = pcvl.Circuit(m)
            # Apply a phase shift on each mode with the same theta.
            for mode in range(m):
                sub.add(mode, pcvl.PS(theta), merge=True)
            return sub

        def BS_block(m, bs_params):
            sub = pcvl.Circuit(m)
            # For demonstration, add a beam splitter between modes 0 and 1.
            sub.add((0, 1), pcvl.BS(), merge=True)
            return sub

        circuit = pcvl.Circuit(m)
        circuit = circuit.add(0, U_block(m, parameters['theta'][0]), merge=True)
        circuit = circuit.add(0, BS_block(m, parameters['alpha']), merge=True)
        circuit = circuit.add(0, U_block(m, parameters['theta'][1]), merge=True)
        circuit = circuit.add(0, BS_block(m, parameters['beta']), merge=True)
        circuit = circuit.add(0, U_block(m, parameters['theta'][2]), merge=True)
        return circuit


class BaseInterferometerBuilder:
    def create_circuit(self, m: int, parameters: list = None) -> pcvl.Circuit:
        """
        Create an alternative interferometer circuit using a simplified rectangular layout.
        If no parameters are provided, generate a list of symbolic parameters of length m*(m-1).
        The lambda uses these parameters in pairs.
        """
        # Compute the expected total number of phases (should be m*(m-1))
        total_needed = m * (m - 1)
        if parameters is None:
            parameters = [pcvl.P(f"phi_{i}") for i in range(total_needed)]

        def base_lambda(i):
            # Use the actual length of parameters to avoid index errors.
            # Each pair of parameters is used for one layer.
            num_pairs = len(parameters) // 2
            j = i % num_pairs
            return (pcvl.BS()
                    .add(0, pcvl.PS(parameters[2 * j]), merge=True)
                    .add(0, pcvl.BS(), merge=True)
                    .add(0, pcvl.PS(parameters[2 * j + 1]), merge=True))

        circuit = pcvl.GenericInterferometer(m, base_lambda)
        return circuit



class ConvolutionalInterferometerBuilder:
    def __init__(self, filterA, filterB, image_size=(28, 28)):
        """
        Initialize with filterA and filterB (numpy arrays) and the expected image size.
        For example, use filterA and filterB of shape (6,5) to yield 23x24=552 outputs.
        """
        self.filterA = filterA
        self.filterB = filterB
        self.image_size = image_size

    def create_circuit(self, m: int, parameters: list = None) -> pcvl.Circuit:
        """
        Constructs a full circuit that interleaves trainable U-blocks with convolutional encoding blocks.
        The conv encoding blocks use parameters for α and β.
        If 'parameters' is None, symbolic parameters are used for the conv blocks.
        Otherwise, 'parameters' is assumed to be a list containing [alpha_flat, beta_flat] concatenated.
        """
        expected_length = m * (m - 1)  # For a triangular interferometer

        # Expected output sizes from valid convolution:
        filterA_shape = self.filterA.shape  # e.g., (6,5)
        filterB_shape = self.filterB.shape  # e.g., (6,5)
        alpha_rows = self.image_size[0] - filterA_shape[0] + 1
        alpha_cols = self.image_size[1] - filterA_shape[1] + 1
        beta_rows  = self.image_size[0] - filterB_shape[0] + 1
        beta_cols  = self.image_size[1] - filterB_shape[1] + 1
        alpha_out_len = alpha_rows * alpha_cols  # e.g., 23*24 = 552
        beta_out_len  = beta_rows * beta_cols

        if parameters is None:
            # Use symbolic parameters for the conv blocks.
            alpha_params = [pcvl.P(f"alpha_{i}") for i in range(alpha_out_len)]
            beta_params  = [pcvl.P(f"beta_{j}") for j in range(beta_out_len)]
        else:
            # Split the provided parameter list into alpha and beta portions.
            alpha_params = parameters[:alpha_out_len]
            beta_params  = parameters[alpha_out_len:alpha_out_len+beta_out_len]

        # Truncate the conv parameters to the expected length.
        if len(alpha_params) > expected_length:
            alpha_params = alpha_params[:expected_length]
        if len(beta_params) > expected_length:
            beta_params = beta_params[:expected_length]
        # Optionally, pad if fewer (not likely with full convolution)
        if len(alpha_params) < expected_length:
            alpha_params.extend([0.0]*(expected_length - len(alpha_params)))
        if len(beta_params) < expected_length:
            beta_params.extend([0.0]*(expected_length - len(beta_params)))

        # Build sub-circuits:
        U1 = pcvl.GenericInterferometer(m, lambda idx: pcvl.BS() // pcvl.PS(pcvl.P(f"theta1_{idx}")), shape=pcvl.InterferometerShape.TRIANGLE)
        U2 = pcvl.GenericInterferometer(m, lambda idx: pcvl.BS() // pcvl.PS(pcvl.P(f"theta2_{idx}")), shape=pcvl.InterferometerShape.TRIANGLE)
        U3 = pcvl.GenericInterferometer(m, lambda idx: pcvl.BS() // pcvl.PS(pcvl.P(f"theta3_{idx}")), shape=pcvl.InterferometerShape.TRIANGLE)
        Conv1 = pcvl.GenericInterferometer(m, lambda idx: pcvl.BS() // pcvl.PS(alpha_params[idx]), shape=pcvl.InterferometerShape.TRIANGLE)
        Conv2 = pcvl.GenericInterferometer(m, lambda idx: pcvl.BS() // pcvl.PS(beta_params[idx]), shape=pcvl.InterferometerShape.TRIANGLE)

        # Compose the full circuit by merging the sub-circuits.
        full_circuit = compose_circuits([U1, Conv1, U2, Conv2, U3])
        return full_circuit



    def compute_conv_parameters(self, image_tensor: np.ndarray) -> list:
        """
        Given an input image (as a numpy array), compute the convolution outputs for filterA and filterB,
        normalize them to the [0, 2π] range, and return the concatenated phase parameter list.
        If the input is 1D, reshape it to 2D using a factorization of its length.
        """
        if image_tensor.ndim != 2:
            if image_tensor.ndim == 1:
                # Import the downsample shape helper from utils
                from utils import compute_downsample_shape
                h, w = compute_downsample_shape(image_tensor.size)
                image_tensor = image_tensor.reshape((h, w))
            else:
                raise ValueError("Input image_tensor must be 2D after squeezing.")

        # Compute valid convolutions.
        alpha = convolve2d(image_tensor, self.filterA, mode="valid")
        beta  = convolve2d(image_tensor, self.filterB, mode="valid")
        # Flatten and normalize: here we use modulo 1 and scale to 2π (adjust normalization as needed)
        alpha_flat = (alpha.flatten() % 1.0) * 2 * np.pi
        beta_flat  = (beta.flatten() % 1.0) * 2 * np.pi
        param_vec = np.concatenate([alpha_flat, beta_flat])
        return param_vec.tolist()





class BosonSampler:
    def __init__(self, m: int, n: int, postselect: int = None, backend: str = "SLOS", session=None, builder: InterferometerBuilder = None, cache_enabled=False, cache_directory="results/cache"):
        """
        Photonic boson sampler for embedding data.
        :param m: number of modes
        :param n: number of photons (n <= m)
        :param postselect: minimum detected photons for a valid output (defaults to n if None)
        :param backend: local simulation backend name (ignored if session is provided)
        :param session: optional Perceval session for remote execution (Scaleway)
        :param builder: An instance of InterferometerBuilder to build the circuit.
        """
        assert n <= m, "Photons n must be <= modes m"
        self.m = m
        self.n = n
        self.postselect = postselect if postselect is not None else n
        assert self.postselect <= n, "Postselect cannot exceed number of photons"
        self.backend = backend
        self.session = session  # If not None, use remote session for simulation
        # If no builder is given, use the triangular mesh as default.
        if builder is None:
            self.builder = TriangularInterferometerBuilder()
        else:
            self.builder = builder

        # Caching settings
        self.cache_enabled = cache_enabled
        self.cache_directory = cache_directory
        if self.cache_enabled and not os.path.exists(self.cache_directory):
            os.makedirs(self.cache_directory)


    def _compute_hash(self, data_tensor):
        # Compute a unique hash based on the tensor contents.
        # Ensure the tensor is on CPU and convert to a numpy byte string.
        return hashlib.md5(data_tensor.cpu().numpy().tobytes()).hexdigest()


    @property
    def nb_parameters(self) -> int:
        """Number of phase parameters available (used for embedding)."""
        return self.m * (self.m - 1) - (self.m // 2)


    @property
    def embedding_size(self) -> int:
        """Number of output features (possible output states after postselection)."""
        from math import comb
        size = 0
        for k in range(self.postselect, self.n + 1):
            size += comb(self.m, k)
        return size


    def create_circuit(self, parameters: list[float] = None) -> pcvl.Circuit:
        """Delegates the creation of the circuit to the builder."""
        return self.builder.create_circuit(self.m, parameters)

    # TODO... DELTE
    # def create_circuit(self, parameters: list[float] = None) -> pcvl.Circuit:
    #     """
    #     Create an interferometer circuit with given phase parameters.
    #     If no parameters are provided, uses symbolic placeholders.
    #     """
    #     # (Use Perceval's GenericInterferometer for a mesh of beamsplitters and phase shifters)
    #     if parameters is None:
    #         # Use symbolic parameters if none provided
    #         params = [pcvl.P(f"phi_{i}") for i in range(self.m * (self.m - 1))]
    #         # Duplicate each parameter for paired phase shifters
    #         #params = [val for p in parameters for val in (p, p)]
    #     else:
    #         params = parameters
    #     return pcvl.GenericInterferometer(self.m, lambda idx: (
    #                 pcvl.BS()  # 50:50 beam splitter
    #                 .add(0, pcvl.PS(params[2*idx]))   # phase shifter, then another BS
    #                 .add(0, pcvl.BS())
    #                 .add(0, pcvl.PS(params[2*idx+1]))
    #            ))

    def prepare_processor(self, processor: pcvl.Processor, parameters: list[float]):
        """Configure the given processor with the circuit, input state, and detection filters."""
        processor.set_circuit(self.create_circuit(parameters))
        processor.min_detected_photons_filter(self.postselect)  # enforce minimum detected photons
        #processor.thresholded_output(True)   # <- DEPRECATED # use threshold detectors (no photon-number resolution)


        from perceval.components import Detector
        for mode in range(self.m):
            processor.add(mode, Detector.threshold())



        # Prepare an input state with n photons evenly spaced in m modes (e.g., [1,0,1,0,...] for 2 photons)
        input_state = [0] * self.m
        if self.n > 0:
            step = max(1, self.m // self.n)
            #step = self.m // self.n
            for i in range(self.n):
                input_state[i * step] = 1
        processor.with_input(pcvl.BasicState(input_state))

    def run(self, parameters: list[float], n_samples: int):
        """
        Run the boson sampler circuit with given phase parameters, sample n_samples times.
        Returns a Perceval BSDistribution (a dict-like object of outcome probabilities).
        """
        if self.session is not None:
            # Build a remote processor via the session (e.g., Scaleway QPU or simulator)
            proc = self.session.build_remote_processor()            # Remote QPU simulation
        else:
            proc = pcvl.Processor(self.backend, self.m)             # Local simulation backend
        # Configure the processor with the circuit and input state
        self.prepare_processor(proc, parameters)
        # Use Perceval's sampling algorithm (Sampler) to run the circuit
        sampler = pcvl.algorithm.Sampler(proc, max_shots_per_call=n_samples)
        job = sampler.probs(n_samples)  # run sampling
        # If job has the method get_results(), use it; otherwise, assume job is already the result.
        if hasattr(job, "get_results"):
            result = job.get_results() # or job.execute_sync() depending on your workflow
        else:
            result = job

        return result  # BSDistribution of outcomes


    def _distribution_to_feature_vector(self, distribution):
        """
        Convert a distribution dictionary into a fixed-length feature vector.
        """
        feature_vec = torch.zeros(self.embedding_size)
        state_list = self._generate_all_output_states()  # list of BasicState outcomes in canonical order
        for i, state in enumerate(state_list):
            feature_vec[i] = distribution.get(state, 0.0)
        return feature_vec

    def adaptive_embed(self, data_tensor, min_samples=100, max_samples=None, tol=1e-3):
        """
        Compute the quantum embedding adaptively.
        Starts with min_samples and doubles until the feature vector converges
        (norm difference < tol) or until max_samples is reached.
        """
        if max_samples is None:
            max_samples = self.n_samples  # default maximum samples from config

        prev_embedding = None
        samples = min_samples
        while samples <= max_samples:
            if hasattr(self, "params"):
                distribution = self.run(n_samples=samples)
            else:
                distribution = self.run(parameters=[], n_samples=samples)
            current_embedding = self._distribution_to_feature_vector(distribution)
            if prev_embedding is not None:
                diff = torch.norm(current_embedding - prev_embedding).item()
                if diff < tol:
                    print(f"Adaptive sampling converged with {samples} samples.")
                    return current_embedding
            prev_embedding = current_embedding
            samples *= 2
        print(f"Adaptive sampling reached max samples: {max_samples}")
        return prev_embedding

    # def adaptive_embed(self, data_tensor, min_samples=100, max_samples=None, tol=1e-3):
    #     """
    #     Compute the quantum embedding adaptively.
    #     It computes a feature vector (from the output distribution) and doubles the sample count
    #     until the difference between successive feature vectors is below tol or until max_samples is reached.
    #     """
    #     if max_samples is None:
    #         max_samples = self.n_samples  # default maximum samples from config

    #     # Prepare phase parameters (same as in embed()):
    #     flat = data_tensor.flatten()
    #     if flat.shape[0] > self.nb_parameters:
    #         raise ValueError("Input tensor too large for the number of modes/photons")
    #     phases = torch.zeros(self.nb_parameters)
    #     phases[:flat.shape[0]] = flat
    #     phase_list = (phases * 2 * torch.pi).tolist()

    #     prev_feature = None
    #     samples = min_samples
    #     while samples <= max_samples:
    #         # Run the circuit with the computed phase_list.
    #         distribution = self.run(phase_list, n_samples=samples)
    #         # Convert distribution to a feature vector:
    #         feature_vec = torch.zeros(self.embedding_size)
    #         state_list = self._generate_all_output_states()  # list of BasicState outcomes in canonical order
    #         for i, state in enumerate(state_list):
    #             feature_vec[i] = distribution.get(state, 0.0)
    #         # Compare with the previous feature vector.
    #         if prev_feature is not None:
    #             diff = torch.norm(feature_vec - prev_feature).item()
    #             print(f"Adaptive sampling: {samples} samples, diff = {diff:.6f}")
    #             if diff < tol:
    #                 print(f"Adaptive sampling converged with {samples} samples.")
    #                 return feature_vec
    #         prev_feature = feature_vec
    #         samples *= 2
    #     print(f"Adaptive sampling reached max samples: {max_samples}")
    #     return prev_feature


    # TODO delete
    # def adaptive_embed(self, data_tensor, min_samples=100, max_samples=None, tol=1e-3):
    #     """
    #     Compute the quantum embedding adaptively.
    #     Starts with min_samples and doubles until the embedding converges
    #     (difference less than tol) or until max_samples is reached.
    #     """
    #     if max_samples is None:
    #         max_samples = self.n_samples  # default maximum samples from config

    #     prev_embedding = None
    #     samples = min_samples
    #     while samples <= max_samples:
    #         current_embedding = self.run(parameters=[], n_samples=samples)
    #         if prev_embedding is not None:
    #             diff = torch.norm(current_embedding - prev_embedding).item()
    #             if diff < tol:
    #                 print(f"Adaptive sampling converged with {samples} samples.")
    #                 return current_embedding
    #         prev_embedding = current_embedding
    #         samples *= 2
    #     print(f"Adaptive sampling reached max samples: {max_samples}")
    #     return prev_embedding



    def embed(self, data_tensor, n_samples: int):
        """
        Embed an input tensor (image data) into a quantum feature vector.
        The input tensor values (assumed in [0,1]) are mapped to phase shifts.
        :param data_tensor: input torch tensor with values in [0,1]
        :param n_samples: number of samples for Monte Carlo estimation (ignored if using an analytic mode)
        :return: torch tensor of shape (embedding_size,) with estimated output probabilities.
        """

        # If caching is enabled, try to load the cached embedding.
        if getattr(self, "cache_enabled", False):
            key = hashlib.md5(data_tensor.cpu().numpy().tobytes()).hexdigest()
            cache_file = os.path.join(self.cache_directory, f"{key}.pkl")
            if os.path.exists(cache_file):
                print("Loading embedding from cache.")
                with open(cache_file, "rb") as f:
                    return pickle.load(f)


     # Check if using convolutional embedding.
        if isinstance(self.builder, ConvolutionalInterferometerBuilder):
            # Convert data_tensor to numpy array (assume shape (1,28,28) or (28,28))
            img_np = data_tensor.squeeze().cpu().numpy()
            phase_list = self.builder.compute_conv_parameters(img_np)
        else:
            # Compute embedding if not cached.
            flat = data_tensor.flatten()
            if flat.shape[0] > self.nb_parameters:
                raise ValueError("Input tensor too large for the number of modes/photons")
            # Prepare phase parameters: pad/truncate to fill the interferometer.
            # Pad or truncate the phases vector to fill all required phase shifters
            #self.nb_parameters == phase_count = self.m * (self.m - 1)  # number of phase parameters needed for full interferometer
            phases = torch.zeros(self.nb_parameters)
            # Use data values (scaled 0-1) for the first part of phases, remaining stay 0
            phases[:flat.shape[0]] = flat
            # Scale phases from [0,1] to [0, 2π] as phase shifts
            phase_list = (phases * 2 * torch.pi).tolist()


        # For variational sampler, self.run expects only n_samples.
        if hasattr(self, "params"):
            distribution = self.run(n_samples=n_samples)
        else:
            distribution = self.run(parameters=phase_list, n_samples=n_samples)

        ## Run the circuit and get the output distribution
        ##distribution = self.run(phase_list, n_samples)


        # Convert distribution to a fixed-length probability vector
        #feature_vec = self._distribution_to_feature_vector(distribution)
        feature_vec = torch.zeros(self.embedding_size)
        state_list = self._generate_all_output_states()  # list of BasicState outcomes in canonical order
        for i, state in enumerate(state_list):
            feature_vec[i] = distribution.get(state, 0.0)


        # Save the computed embedding to cache.
        if getattr(self, "cache_enabled", False):
            with open(cache_file, "wb") as f:
                pickle.dump(feature_vec, f)


        return feature_vec

    #@lru_cache(maxsize=1)
    def _generate_all_output_states(self):
        """Precompute all possible output basis states (with postselection).
        Generate and cache all possible output states (as BasicState objects).
        """
        states = []
        for k in range(self.postselect, self.n + 1):
            from itertools import combinations
            for ones in combinations(range(self.m), k):
                bitstring = [1 if j in ones else 0 for j in range(self.m)]
                states.append(pcvl.BasicState(bitstring))
        return states

    from concurrent.futures import ThreadPoolExecutor

    @staticmethod
    def parallel_embed(self, image_list, n_samples, adaptive=False, min_samples=100, tol=1e-3):
        """
        Compute embeddings for a list of images concurrently.
        NOTE: This method is useful for local simulation.
            However, QaaS (remote) does not handle batch processing well.
            It is recommended to process images sequentially when using the remote platform.
        Parameters:
          image_list : List of image tensors.
          n_samples  : Maximum number of samples for each embedding.
          adaptive   : If True, uses adaptive_embed; otherwise uses embed.
          min_samples: Starting sample count for adaptive sampling.
          tol        : Tolerance for convergence in adaptive sampling.
        Returns:
          A list of embeddings.
        """
        with ThreadPoolExecutor(max_workers=4) as executor:
            futures = []
            for img in image_list:
                if adaptive:
                    futures.append(executor.submit(self.adaptive_embed, img, min_samples, n_samples, tol))
                else:
                    futures.append(executor.submit(self.embed, img, n_samples))
            embeddings = [f.result() for f in futures]
        return embeddings






class VariationalBosonSampler(BosonSampler):
    def __init__(self, m: int, n: int, postselect: int = None, backend: str = "SLOS",
                 session=None, builder: InterferometerBuilder = None, cache_enabled=False,
                 cache_directory="results/cache", init_params=None):
        """
        A variant of BosonSampler where the phase parameters are trainable.
        """
        super().__init__(m, n, postselect, backend, session, builder, cache_enabled, cache_directory)
        # Initialize trainable parameters. If not provided, initialize to small constant values.
        if init_params is None:
            init_params = [0.1] * self.nb_parameters
        # Register parameters as a torch.nn.Parameter so they are updated during training.
        self.params = nn.Parameter(torch.tensor(init_params, dtype=torch.float32))

    adaptive_embed = BosonSampler.adaptive_embed


    def create_circuit(self, parameters: list[float] = None) -> pcvl.Circuit:
        """
        Overrides the create_circuit method to use trainable parameters.
        """
        # Use the current trainable parameters from self.params
        phase_list = self.params.tolist()
        return self.builder.create_circuit(self.m, phase_list)

    def run(self, n_samples: int, parameters: list[float] = None):
        """
        Overrides run() to build the circuit using trainable parameters.
        The 'parameters' argument is ignored since self.params is used.
        """
        circuit = self.create_circuit()  # This uses self.params internally.
        if self.session is not None:
            proc = self.session.build_remote_processor()
        else:
            proc = pcvl.Processor(self.backend, self.m)
        # We ignore the passed 'parameters' and use self.params instead.
        self.prepare_processor(proc, parameters=[])
        sampler = pcvl.algorithm.Sampler(proc, max_shots_per_call=n_samples)
        job = sampler.probs(n_samples)
        if hasattr(job, "get_results"):
            result = job.get_results()
        else:
            result = job
        return result




##############
# Enhanced BosonSampler with Caching
# Implementing caching to avoid redundant quantum computations:
############
from functools import lru_cache
import os
import pickle
import hashlib

class CachedBosonSampler(BosonSampler):
    """
    Extended BosonSampler with caching capabilities to avoid redundant computations.
    """
    def __init__(self, m, n, postselect=None, session=None,
                 cache_enabled=True, cache_size=1000,
                 disk_cache=False, cache_dir=None):
        super().__init__(m, n, postselect, session)
        self.cache_enabled = cache_enabled
        self.disk_cache = disk_cache
        self.cache_dir = cache_dir

        if disk_cache and cache_dir:
            os.makedirs(cache_dir, exist_ok=True)

        # Use the wrapper function to apply caching
        if cache_enabled:
            self.embed = self._cached_embed(self.embed, maxsize=cache_size)

    def _cached_embed(self, func, maxsize=1000):
        """Apply in-memory caching with LRU strategy."""
        memory_cache = lru_cache(maxsize=maxsize)(func)

        def wrapped_func(t, n_sample):
            # Convert tensor to tuple for hashing
            t_tuple = tuple(t.reshape(-1).tolist())

            # Check disk cache if enabled
            if self.disk_cache and self.cache_dir:
                cache_key = hashlib.md5(str(t_tuple).encode()).hexdigest()
                cache_file = os.path.join(self.cache_dir, f"{cache_key}.pkl")

                if os.path.exists(cache_file):
                    with open(cache_file, 'rb') as f:
                        return pickle.load(f)

            # Use in-memory cache or compute
            result = memory_cache(t, n_sample)

            # Save to disk cache if enabled
            if self.disk_cache and self.cache_dir:
                with open(cache_file, 'wb') as f:
                    pickle.dump(result, f)

            return result

        return wrapped_func if self.cache_enabled else func

In [60]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class MnistModel(nn.Module):
    def __init__(self, device="cpu", embedding_size=0):
        """
        A simple MNIST classifier.
        If embedding_size > 0, the model expects an additional quantum embedding vector to be concatenated with the flattened image.
        """
        super().__init__()
        input_size = 28 * 28
        if embedding_size:
            input_size += embedding_size
        self.device = device
        self.embedding_size = embedding_size
        self.linear = nn.Linear(input_size, 10)

    def forward(self, xb, emb=None):
        xb = xb.view(-1, 784)
        if self.embedding_size and emb is not None:
            xb = torch.cat((xb, emb), dim=1)
        out = self.linear(xb)
        return out

    def training_step(self, batch, emb=None):
        images, labels = batch
        images, labels = images.to(self.device), labels.to(self.device)
        output = self(images, emb.to(self.device)) if self.embedding_size and emb is not None else self(images)
        loss = F.cross_entropy(output, labels)
        acc = accuracy(output, labels)
        return loss, acc

    def validation_step(self, batch, emb=None):
        images, labels = batch
        images, labels = images.to(self.device), labels.to(self.device)
        output = self(images, emb.to(self.device)) if self.embedding_size and emb is not None else self(images)
        loss = F.cross_entropy(output, labels)
        acc = accuracy(output, labels)
        return {"val_loss": loss, "val_acc": acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x["val_loss"] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        batch_accs = [x["val_acc"] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()
        return {"val_loss": epoch_loss.item(), "val_acc": epoch_acc.item()}

    def epoch_end(self, epoch, result):
        print(f"Epoch [{epoch}], val_loss: {result['val_loss']:.4f}, val_acc: {result['val_acc']:.4f}")
        return result["val_loss"], result["val_acc"]




class HybridMnistModel(nn.Module):
    def __init__(self, device="cpu", embedding_size=0):
        """
        A hybrid model that first extracts CNN features from the image and then
        concatenates the quantum embedding.
        """
        super().__init__()
        # Define a simple CNN feature extractor.
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),  # input: [B,1,28,28] -> [B,16,28,28]
            nn.ReLU(),
            nn.MaxPool2d(2),  # -> [B,16,14,14]
            nn.Conv2d(16, 32, kernel_size=3, padding=1),  # -> [B,32,14,14]
            nn.ReLU(),
            nn.MaxPool2d(2)   # -> [B,32,7,7]
        )
        cnn_output_size = 32 * 7 * 7
        total_input_size = cnn_output_size + embedding_size
        self.fc = nn.Linear(total_input_size, 10)
        self.device = device
        self.embedding_size = embedding_size

    def forward(self, image, emb=None):
        # Process image with CNN.
        features = self.cnn(image)  # Expecting image shape: [B,1,28,28]
        features = features.view(features.size(0), -1)
        if emb is not None:
            features = torch.cat((features, emb), dim=1)
        return self.fc(features)

    def training_step(self, batch, emb=None):
        images, labels = batch
        images, labels = images.to(self.device), labels.to(self.device)
        output = self(images, emb.to(self.device)) if self.embedding_size and emb is not None else self(images)
        loss = F.cross_entropy(output, labels)
        _, preds = torch.max(output, dim=1)
        acc = torch.tensor(torch.sum(preds == labels).item() / len(preds))
        return loss, acc

    def validation_step(self, batch, emb=None):
        images, labels = batch
        images, labels = images.to(self.device), labels.to(self.device)
        output = self(images, emb.to(self.device)) if self.embedding_size and emb is not None else self(images)
        loss = F.cross_entropy(output, labels)
        _, preds = torch.max(output, dim=1)
        acc = torch.tensor(torch.sum(preds == labels).item() / len(preds))
        return {"val_loss": loss, "val_acc": acc}

    def epoch_end(self, epoch, result):
        print(f"Epoch [{epoch}], val_loss: {result['val_loss']:.4f}, val_acc: {result['val_acc']:.4f}")
        return result["val_loss"], result["val_acc"]

In [61]:
import os
import pandas as pd
import re
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt


#####################
#   DATA CONFIG     #
#####################

# previously src/config.py
import yaml

def load_config(config_path="yaml_default_config.yaml"):
    """
    Load configuration from the YAML file.
    """
    base_dir = os.getcwd()
    config_full_path = os.path.join(base_dir, config_path)
    with open(config_full_path, "r") as f:
        cfg = yaml.safe_load(f)
    return cfg

def create_scaleway_session(cfg):
    """
    Create a Scaleway QaaS session based on the config and environment variables.
    Expects SCW_PROJECT_ID and SCW_SECRET_KEY to be set in the environment.
    """
    import perceval.providers.scaleway as scw
    # Get credentials from the .env file using python-decouple.
    proj_id = decouple_config("SCW_PROJECT_ID") #
    token = decouple_config("SCW_SECRET_KEY") #
    platform = cfg["quantum"].get("scaleway_platform", "sim:sampling:p100")
    if not proj_id or not token:
        raise RuntimeError("Scaleway credentials not found in environment.")
    session = scw.Session(
        project_id=proj_id,
        token=token,
        platform=platform,
        max_idle_duration_s=1200,
        max_duration_s=3600
    )
    return session






import os


#####################
#   DATA UTILS      #
#####################
class MNIST_partial(Dataset):
    def __init__(self, data="./data", transform=None, split="train"):
        """
        Load the reduced MNIST dataset from CSV.
        :param data: Path to folder containing train.csv and val.csv.
        :param transform: Optional transform function.
        :param split: "train" or "val"
        """
        # Compute the base directory (directory of this file)
        base_dir = os.getcwd()
        # If 'data' is given as a relative path, make it absolute relative to this file:
        self.data_dir = os.path.join(base_dir, data)
        self.transform = transform
        if split == "train":
            filename = os.path.join(self.data_dir, "train.csv")
        elif split == "val":
            filename = os.path.join(self.data_dir, "val.csv")
        else:
            raise AttributeError("split must be 'train' or 'val'")
        self.df = pd.read_csv(filename)

    def __len__(self):
        return len(self.df["image"])

    def __getitem__(self, idx):
        img_str = self.df["image"].iloc[idx]
        label = int(self.df["label"].iloc[idx])
        img_list = re.split(r",", img_str)
        img_list[0] = img_list[0][1:]
        img_list[-1] = img_list[-1][:-1]
        img_float = [float(el) for el in img_list]
        # Reshape to (1, 28, 28)
        img_tensor = torch.tensor(img_float).reshape(1, 28, 28)
        if self.transform is not None:
            img_tensor = self.transform(img_tensor)
        return img_tensor, label

def get_dataloader(dataset, batch_size, shuffle):
    """Return a DataLoader for the given dataset."""
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

#####################
# TRAINING UTILS    #
#####################
def plot_training_metrics(train_acc, val_acc, train_loss, val_loss):
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    epochs = list(range(1, len(train_acc) + 1))
    axes[0].plot(epochs, train_acc, label="Training Accuracy")
    axes[0].plot(epochs, val_acc, label="Validation Accuracy")
    axes[0].set_xlabel("Epochs")
    axes[0].set_ylabel("Accuracy")
    axes[0].set_title("Accuracy over Epochs")
    axes[0].legend()
    axes[1].plot(epochs, train_loss, label="Training Loss")
    axes[1].plot(epochs, val_loss, label="Validation Loss")
    axes[1].set_xlabel("Epochs")
    axes[1].set_ylabel("Loss")
    axes[1].set_title("Loss over Epochs")
    axes[1].legend()
    plt.tight_layout()
    plt.savefig("training_curves.png")
    plt.close()

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


def compute_downsample_shape(nb_params, target_aspect=1.0):
    """
    Compute a downsample shape (height, width) such that height * width == nb_params if possible.
    If not, choose the factor pair that is as close as possible to a square (or the target aspect ratio).
    target_aspect=1.0 corresponds to a square.

    Parameters:
      nb_params: int - the desired total number of pixels (phase parameters)
      target_aspect: float - desired aspect ratio (width/height), default 1.0

    Returns:
      (height, width): tuple of ints representing the downsample shape.
    """
    from math import sqrt
    # Try to find factor pairs exactly.
    for h in range(int(sqrt(nb_params)), 0, -1):
        if nb_params % h == 0:
            w = nb_params // h
            return h, w
    # If no factor pair exactly divides nb_params, default to a square shape.
    h = int(sqrt(nb_params))
    return h, h



######################
# PCA Downsampling   #
######################


from sklearn.decomposition import PCA
import numpy as np
import torch

def pca_downsample(image_tensor, pca_model):
    """
    Downsample a 28x28 image using a pre-trained PCA model.

    Parameters:
      image_tensor: torch tensor of shape [1, 28, 28] (or [28,28])
      pca_model: a scikit-learn PCA model pre-trained on flattened MNIST images

    Returns:
      A torch tensor of shape [pca_model.n_components]
    """
    # Ensure the image is in 2D: if [1,28,28], remove the channel dimension.
    if image_tensor.dim() == 3 and image_tensor.shape[0] == 1:
        image_tensor = image_tensor.squeeze(0)
    # Flatten the image (28*28)
    img_flat = image_tensor.view(-1).cpu().numpy()
    # Project the flattened image onto the PCA components
    reduced = pca_model.transform([img_flat])[0]
    # Convert back to a torch tensor
    return torch.tensor(reduced, dtype=torch.float32)










import torch.fft

def fourier_encode(image_tensor, n_features=100):
    """
    Computes Fourier features from an image tensor.

    Parameters:
      image_tensor: A tensor of shape [1, 28, 28] (or [28,28]).
      n_features  : Number of Fourier features to extract.

    Returns:
      A 1D tensor of Fourier features (absolute values of the FFT).
    """
    # Ensure the image is 2D.
    if image_tensor.dim() == 3 and image_tensor.shape[0] == 1:
        image_tensor = image_tensor.squeeze(0)
    fft_result = torch.fft.fft2(image_tensor)
    fft_abs = torch.abs(fft_result).flatten()
    # Select the first n_features; you might also choose to sort or use a different selection method.
    return fft_abs[:n_features]






In [51]:
import os
import torch
import yaml
import pickle
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm


# A simple PCA-based classifier
class ClassicPCAClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_classes=10):
        super(ClassicPCAClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# Function to load and prepare data (flattening the images)
def prepare_pca_data(dataset):
    data_list, labels_list = [], []
    for img, label in dataset:
        data_list.append(img.view(-1).numpy())
        labels_list.append(label)
    return np.array(data_list), np.array(labels_list)

# Minimal main for classical baseline
def main_classic():
    result_dir = f"results/classical_run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    os.makedirs(result_dir, exist_ok=True)
    print(f"Results will be saved to: {result_dir}")

    # Load datasets from CSV files in "data" folder
    train_dataset = MNIST_partial(data="data", split="train")
    val_dataset = MNIST_partial(data="data", split="val")

    # Prepare data for PCA
    X_train, y_train = prepare_pca_data(train_dataset)
    X_val, y_val = prepare_pca_data(val_dataset)

    # For matching quantum parameter count, suppose n_components is:
    m = cfg["quantum"]["n_modes"]
    n_components = m * (m - 1) - (m // 2)
    print(f"Using PCA with {n_components} components")

    from sklearn.decomposition import PCA
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_val_pca = pca.transform(X_val)

    # Build datasets for PCA features
    class PCADataset(torch.utils.data.Dataset):
        def __init__(self, X, y):
            self.X = torch.tensor(X, dtype=torch.float32)
            self.y = torch.tensor(y, dtype=torch.long)
        def __len__(self):
            return len(self.X)
        def __getitem__(self, idx):
            return self.X[idx], self.y[idx]

    train_dataset_pca = PCADataset(X_train_pca, y_train)
    val_dataset_pca = PCADataset(X_val_pca, y_val)

    batch_size = cfg["data"]["batch_size"]
    train_loader = DataLoader(train_dataset_pca, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset_pca, batch_size=batch_size, shuffle=False)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Using device:", device)

    # Initialize classical model
    model = ClassicPCAClassifier(input_dim=n_components, hidden_dim=64, num_classes=10).to(device)

    optimizer = optim.Adam(model.parameters(), lr=cfg["training"]["learning_rate"])
    criterion = nn.CrossEntropyLoss()

    epochs = cfg["training"]["epochs"]
    for epoch in range(epochs):
        model.train()
        running_loss, running_corrects, num_samples = 0.0, 0, 0
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels).item()
            num_samples += inputs.size(0)
        epoch_loss = running_loss / num_samples
        epoch_acc = running_corrects / num_samples
        print(f"Epoch {epoch+1} - Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}")

    # (Optional: add evaluation and visualizations, e.g. confusion matrix, PCA component plots, etc.)
    # Save the PCA model for later use
    with open(os.path.join("data", "pca_model.pkl"), "wb") as f:
        pickle.dump(pca, f)
    print("PCA model saved.")

main_classic()

Results will be saved to: results/classical_run_20250331_175823
Using PCA with 540 components
Using device: cuda


Epoch 1/5: 100%|██████████| 200/200 [00:01<00:00, 194.83it/s]


Epoch 1 - Loss: 1.3210, Acc: 0.7633


Epoch 2/5: 100%|██████████| 200/200 [00:00<00:00, 589.28it/s]


Epoch 2 - Loss: 0.3654, Acc: 0.9173


...

Epoch 3 - Loss: 0.2334, Acc: 0.9407


Epoch 4/5: 100%|██████████| 200/200 [00:00<00:00, 585.54it/s]


Epoch 4 - Loss: 0.1734, Acc: 0.9557


Epoch 5/5: 100%|██████████| 200/200 [00:00<00:00, 555.50it/s]

Epoch 5 - Loss: 0.1315, Acc: 0.9698
PCA model saved.


In [ ]:
# --- Begin Minimal Hybrid Main ---
def main_quantum():
    print("Loaded inline configuration:")
    print(cfg)

    # For a local simulation, we do not create a Scaleway session.
    session = None

    circuit_type = cfg["quantum"].get("circuit_type", "triangular")
    if circuit_type == "pdf":
        builder = PdfInterferometerBuilder()
    elif circuit_type == "base":
        builder = BaseInterferometerBuilder()
    elif circuit_type == "triangular":
        builder = TriangularInterferometerBuilder()
    elif circuit_type == "rectangular":
        builder = RectangularInterferometerBuilder()
    elif circuit_type == "convolutional":
        # Define random filters for demonstration.
        filterA = np.random.rand(6, 5)
        filterB = np.random.rand(6, 5)
        builder = ConvolutionalInterferometerBuilder(filterA, filterB, image_size=(28,28))
    else:
        raise ValueError(f"Unsupported circuit type: {circuit_type}")

    # Instantiate the boson sampler (choose variational or not)
    if cfg["quantum"].get("variational", False):
        bs = VariationalBosonSampler(
            m=cfg["quantum"]["n_modes"],
            n=cfg["quantum"]["n_photons"],
            postselect=cfg["quantum"]["postselect"],
            backend=cfg["quantum"]["local_backend"],
            session=session,
            builder=builder,
            cache_enabled=cfg["quantum"].get("cache_enabled", False),
            cache_directory=cfg["quantum"].get("cache_directory", "results/cache")
        )
    else:
        bs = BosonSampler(
            m=cfg["quantum"]["n_modes"],
            n=cfg["quantum"]["n_photons"],
            postselect=cfg["quantum"]["postselect"],
            backend=cfg["quantum"]["local_backend"],
            session=session,
            builder=builder,
            cache_enabled=cfg["quantum"].get("cache_enabled", False),
            cache_directory=cfg["quantum"].get("cache_directory", "results/cache")
        )
    bs.n_samples = cfg["quantum"]["n_samples"]

    print("BosonSampler embedding size:", bs.embedding_size)

    # Load dataset (assume MNIST CSVs are in the "data" folder)
    train_dataset = MNIST_partial(data="data", split="train")
    val_dataset = MNIST_partial(data="data", split="val")
    train_loader = get_dataloader(train_dataset, batch_size=cfg["data"]["batch_size"], shuffle=cfg["data"]["shuffle"])
    val_loader = get_dataloader(val_dataset, batch_size=cfg["data"]["batch_size"], shuffle=False)

    # Choose device
    device = "cuda" if (cfg["training"]["device"]=="auto" and torch.cuda.is_available()) else "cpu"
    print("Using device:", device)

    # Initialize the model (Hybrid model uses the quantum embedding)
    net = HybridMnistModel(device=device, embedding_size=bs.embedding_size)
    net = net.to(device)

    # Setup optimizer (include boson sampler parameters if variational)
    optimizer_name = cfg["training"]["optimizer"].lower()
    if cfg["quantum"].get("variational", False):
        params_to_opt = list(net.parameters()) + [bs.params]
    else:
        params_to_opt = net.parameters()

    if optimizer_name == "adam":
        optimizer = torch.optim.Adam(params_to_opt, lr=cfg["training"]["learning_rate"])
    elif optimizer_name == "sgd":
        optimizer = torch.optim.SGD(params_to_opt, lr=cfg["training"]["learning_rate"])
    else:
        raise ValueError("Unsupported optimizer specified in config.")

    # Train the model
    history = fit(
        epochs=cfg["training"]["epochs"],
        lr=cfg["training"]["learning_rate"],
        model=net,
        train_loader=train_loader,
        val_loader=val_loader,
        bs=bs,
        optimizer=optimizer,
        cfg=cfg
    )
    print("Training completed.")

main_quantum()
# --- End Minimal Hybrid Main ---

Loaded inline configuration:
{'data': {'train_csv': 'data/train.csv', 'test_csv': 'data/val.csv', 'batch_size': 30, 'shuffle': True}, 'model': {'type': 'hybrid', 'fourier_features': 50, 'conv_channels': 16, 'kernel_size': 5, 'output_dim': 30}, 'quantum': {'backend': 'local', 'n_modes': 24, 'n_photons': 10, 'postselect': 3, 'n_samples': 1000, 'circuit_type': 'pdf', 'variational': True, 'adaptive_sampling': False, 'cache_enabled': True, 'cache_directory': 'results/cache', 'local_backend': 'SLOS', 'scaleway_platform': 'sim:sampling:2l4', 'downsample_method': 'pca'}, 'training': {'epochs': 5, 'learning_rate': 0.001, 'optimizer': 'adam', 'device': 'auto'}, 'optimization': {'enabled': True, 'method': 'tpe', 'max_evals': 10, 'space': {'n_modes': [6, 12, 24, 30], 'n_photons': [6, 10], 'learning_rate': [0.0001, 0.001, 0.01]}}}
BosonSampler embedding size: 4540085
Using device: cuda
Downsample shape for images set to: 20x27


Epoch 1/5:   0%|          | 0/200 [00:00<?, ?it/s]